# Classification Structure

At first, I used WAF as the idea to group items, after I noticed it is an effective way to perform grouping or clustering, I decided to generalize it further.
This notebook was used to convert the older `classes.json` file to `new_classifications.json` file.

>Note: This notebook is not going to be used further, it is provided as reference to the thought process.

In [ ]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;
using System.Text.Json; 

## Research Progress

As further code iteration, it came to mind that specific classification is not the best approach. So a more generic classification class was created.
The below is the older one, the new structure is maintained under the `Models` folder of the code.

In [ ]:
using System.Text.Json;
using System.Text.Json.Serialization;

// Represents one "pillar" (e.g., "Reliability") which contains a dictionary
// of subcategory name -> WafItem
public class WafRoot : Dictionary<string, Dictionary<string, WafItem>> {}

// Represents an individual subcategory item (e.g., "Simplicity and efficiency")
public class WafItem
{
    [JsonPropertyName("id")]
    public string Id { get; set; }

    [JsonPropertyName("definition")]
    public string Definition { get; set; }

    // We'll add this array to hold our embedding
    [JsonPropertyName("embedded")]
    public float[] Embedded { get; set; }
}

In [ ]:
using System.Text.Json;

// Suppose your old classification JSON is in "old_classification.json"
string oldJson = File.ReadAllText("classes.json");

// This will load a WafRoot, effectively a Dictionary<string, Dictionary<string, WafItem>>
var wafRoot = JsonSerializer.Deserialize<WafRoot>(oldJson);

// Check if null or anything as needed
if (wafRoot == null)
{
    Console.WriteLine("Failed to load or parse the legacy JSON.");
    return;
}

## Using classes from the Models folder

As `polyglot` notebook are evaluated as script, classes with `namespace` cannot be used, my options were to maintain two classes, or to include the `console.dll` file. The decision is below:

In [ ]:
# r "../bin/Debug/net8.0/console.dll"
using ProductLeaders.console.Models;


## Converting

It is straight forward, loading into the old structure and serializing it to the new one.

In [ ]:
var classificationNodes = new List<ClassificationNode>();

foreach (var (pillarName, subcategories) in wafRoot)
{
    // Create a parent ClassificationNode for the top-level pillar
    var parentNode = new ClassificationNode
    {
        Topic = pillarName,
        // We typically don't have an ID/Definition for the top-level in the legacy data,
        // but you could set them if you wish:
        Id = null,
        Definition = null,
        Embedded = null
    };

    // For each subcategory, create a child node
    foreach (var (subTopicName, wafItem) in subcategories)
    {
        var childNode = new ClassificationNode
        {
            Topic = subTopicName,
            Id = wafItem.Id,
            Definition = wafItem.Definition,
            Embedded = wafItem.Embedded
        };

        parentNode.ChildTopics.Add(childNode);
    }

    classificationNodes.Add(parentNode);
}

Saving to a file:

In [ ]:
var options = new JsonSerializerOptions { WriteIndented = true };
string newFormatJson = JsonSerializer.Serialize(classificationNodes, options);

// Save to file
File.WriteAllText("new_classification.json", newFormatJson);

Console.WriteLine("Conversion complete. See new_classification.json for results.");